In [1]:
!pip install -q stellargraph[demos]==1.2.1

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

numba 0.55.1 requires numpy<1.22,>=1.18, but you'll have numpy 1.22.0 which is incompatible.


In [17]:
!pip install -q openpyxl

In [1]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

2022-02-03 11:36:42.708249: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /ihome/crc/install/cuda-11.1.1/pytorch/1.8.1_gpu/python3.8/lib:/usr/lib64:/ihome/crc/install/cuda/11.1.1/lib64/stubs:/ihome/crc/install/cuda/11.1.1/lib64
2022-02-03 11:36:42.708291: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-02-03 11:36:42.709754: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perfo

In [2]:
!ls

dummy.xlsx	      foo.pdf  GCN-2.ipynb  GCN.ipynb
file_for_network.csv  foo.png  GCN-3.ipynb


In [3]:
from stellargraph import StellarGraph
import json
import pandas as pd
import numpy as np
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.metrics import mean_absolute_error, mean_squared_error

import stellargraph as sg
from stellargraph.mapper import HinSAGELinkGenerator
from stellargraph.layer import HinSAGE, link_regression
from tensorflow.keras import Model, optimizers, losses, metrics

import multiprocessing
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import pandas as pd

In [65]:
xl = pd.ExcelFile("dummy.xlsx")
df = xl.parse("Sheet1", header=None)
df.columns=["pathway","tfs"]
df=df.head()
df

,pathway,tfs
0,acute myeloid leukemia,"['STAT3', 'PRDM1']"
1,adipocytokine signaling pathway,"['STAT3', 'PRDM1']"
2,pancreatic cancer,"['STAT3', 'PRDM1']"
3,jak stat signaling pathway,"['STAT3', 'PRDM1']"
4,acute myeloid leukemia,"['STAT3', 'KLF5']"


In [66]:
df.tfs[1].__class__

str

In [67]:
import ast
for i in range(len(df)):
    x =df.tfs[i]
    x = ast.literal_eval(x)
    x = [n.strip() for n in x]
    df.tfs[i]=x


In [68]:
x.__class__

list

In [69]:
df[['tf1','tf2']] = pd.DataFrame(df.tfs.tolist(), index= df.index)

In [73]:
df.drop(["tfs"],axis=1, inplace=True)
df

,pathway,tf1,tf2
0,acute myeloid leukemia,STAT3,PRDM1
1,adipocytokine signaling pathway,STAT3,PRDM1
2,pancreatic cancer,STAT3,PRDM1
3,jak stat signaling pathway,STAT3,PRDM1
4,acute myeloid leukemia,STAT3,KLF5


In [95]:
df_copy=df
df_tf1=df_copy[["pathway","tf1"]]
df_tf1.columns=["pathway","tf"]
df_tf2=df_copy[["pathway","tf2"]]
df_tf2.columns=["pathway","tf"]
df_tf=df_tf1.append(df_tf2)
df_tf["pathway2"]=df_tf["pathway"]
df_tf=df_tf.set_index("tf")
df_tf

,pathway,pathway2
tf,,
STAT3,acute myeloid leukemia,acute myeloid leukemia
STAT3,adipocytokine signaling pathway,adipocytokine signaling pathway
STAT3,pancreatic cancer,pancreatic cancer
STAT3,jak stat signaling pathway,jak stat signaling pathway
STAT3,acute myeloid leukemia,acute myeloid leukemia
PRDM1,acute myeloid leukemia,acute myeloid leukemia
PRDM1,adipocytokine signaling pathway,adipocytokine signaling pathway
PRDM1,pancreatic cancer,pancreatic cancer
PRDM1,jak stat signaling pathway,jak stat signaling pathway


In [51]:
df.head()

,pathway,tfs
0,acute myeloid leukemia,"['STAT3', 'PRDM1']"
1,adipocytokine signaling pathway,"['STAT3', 'PRDM1']"
2,pancreatic cancer,"['STAT3', 'PRDM1']"
3,jak stat signaling pathway,"['STAT3', 'PRDM1']"
4,acute myeloid leukemia,"['STAT3', 'KLF5']"


In [74]:
square_edges = pd.DataFrame(
    {"source": df.tf1.values, "target":  df.tf2.values}
)
square_edges

,source,target
0,STAT3,PRDM1
1,STAT3,PRDM1
2,STAT3,PRDM1
3,STAT3,PRDM1
4,STAT3,KLF5


In [75]:
square = StellarGraph(edges=square_edges)

In [76]:
print(square.info())

StellarGraph: Undirected multigraph
 Nodes: 3, Edges: 5

 Node types:
  default: [3]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [5]
        Weights: all 1 (default)
        Features: none


In [77]:
square_edges_first_second = square_edges.rename(
    columns={"source": "first", "target": "second"}
)
square_edges_first_second

,first,second
0,STAT3,PRDM1
1,STAT3,PRDM1
2,STAT3,PRDM1
3,STAT3,PRDM1
4,STAT3,KLF5


In [79]:

square_first_second = StellarGraph(
    edges=square_edges_first_second, source_column="first", target_column="second"
)
print(square_first_second.info())

StellarGraph: Undirected multigraph
 Nodes: 3, Edges: 5

 Node types:
  default: [3]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [5]
        Weights: all 1 (default)
        Features: none


In [96]:
square_node_data=df_tf
square_named_node_features = StellarGraph(
    {"corner": square_node_data}, {"line": square_edges}
)
print(square_named_node_features.info())

ValueError: could not convert string to float: 'acute myeloid leukemia'

In [55]:

batch_size = 200
epochs = 20
# Use 70% of edges for training, the rest for testing:
train_size = 0.7
test_size = 0.3

In [59]:
edges_train, edges_test = model_selection.train_test_split(
    square_edges, train_size=train_size, test_size=test_size
)

edgelist_train = list(edges_train[["source", "target"]].itertuples(index=False))
edgelist_test = list(edges_test[["source", "target"]].itertuples(index=False))

labels_train = edges_train["rating"]
labels_test = edges_test["rating"]

KeyError: 'rating'

In [60]:
edges_train

,source,target
170,"['STAT3', 'SALL4']",pancreatic cancer
325,"['FOXM1', 'STAT3', 'HLX']",acute myeloid leukemia
309,"['STAT3', 'TP53', 'HMGA1']",chronic myeloid leukemia
323,"['SOX2', 'STAT3', 'HLX']",pancreatic cancer
7,"['STAT3', 'KLF5']",jak stat signaling pathway
...,...,...
221,"['STAT3', 'NKX2-1', 'HEY1']",jak stat signaling pathway
0,"['STAT3', 'PRDM1']",acute myeloid leukemia
188,"['STAT3', 'ESR2']",jak stat signaling pathway
76,"['STAT3', 'HLX']",adipocytokine signaling pathway
